In [56]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [57]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize

In [67]:
def clean_data(text):
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(' ', text) 
    text = BAD_SYMBOLS_RE.sub('', text)
    text = text.replace('x', '')
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text

In [59]:
df = pd.read_csv("../input/nlp-getting-started/train.csv")
df = df.drop(['id', 'keyword', 'location'], axis = 1)
df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))
df['text'] = df['text'].apply(clean_data)
df['text'] = df['text'].str.replace('\d+', '')
df

In [60]:
max_words = 40000
max_seq_len = 200
embed_size = 100

tokenizer = Tokenizer(num_words=max_words, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['text'].values)
word_index = tokenizer.word_index
print('Number of Unique Tokens:', len(word_index))

X = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(X, maxlen=max_seq_len)

y = np.array(df['target'])

print("X:", X)
print("y:", y)

In [62]:
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.30, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

In [63]:
model=tf.keras.Sequential()

# First Layer
model.add(tf.keras.layers.Embedding(max_words, embed_size, input_shape=(X_train.shape[1],)))

# Hidden Layer
model.add(tf.keras.layers.LSTM(units=128, activation='tanh'))

# Output Layer
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

model.summary()

In [55]:
model.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=10, batch_size=128)

test_loss, test_accuracy = model.evaluate(X_test, Y_test)

print("Test accuracy: {}".format(test_accuracy))

In [70]:
train_acc = [0.6789, 0.6690, 0.7403, 0.9129, 0.9445, 0.9621, 0.9762, 0.9842, 0.9645, 0.9923]
train_loss = [.2083, 0.3089, 0.2367, 0.0677, 0.0457, 0.0312, 0.0207, 0.0137, 0.0329, 0.0069]

In [72]:
plt.plot(range(10), train_acc, label="Accuracy")
plt.plot(range(10), train_loss, label="Loss")
plt.title("Accuracy and Loss of Train Data by Epoch")
plt.xlabel("Epoch")
plt.legend()